In [1]:
print(2+3)
print("hello world!")



StatementMeta(, , -1, SessionStarting, , SessionStarting)

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("Jupyter").getOrCreate()
spark.conf.set(
    "fs.azure.account.key.stadominoslab.dfs.core.windows.net",
    ""
)
spark

StatementMeta(83c0425f-724b-4c97-8e6d-0c593361184b, 2, 24, Finished, Available, Finished)

In [20]:
account = "stadominoslab"
container = "ds-landing"
filename = "devices.csv"
read_path = "abfss://%s@%s.dfs.core.windows.net/%s" % (container, account, filename)
print(read_path)

StatementMeta(83c0425f-724b-4c97-8e6d-0c593361184b, 2, 25, Finished, Available, Finished)

abfss://ds-landing@stadominoslab.dfs.core.windows.net/devices.csv


In [ ]:
events = spark.read.option("header", "true").csv("abfss://dpz-ds-prod-landing@staccdbxds1prodwestus3.dfs.core.windows.net/events.csv").withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))
display(events)

In [21]:

devices = spark.read.option("header", "true").csv(read_path)

devices.show()

StatementMeta(83c0425f-724b-4c97-8e6d-0c593361184b, 2, 26, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling o6234.csv.
: java.nio.file.AccessDeniedException: Operation failed: "This request is not authorized to perform this operation using this permission.", 403, HEAD, https://stadominoslab.dfs.core.windows.net/ds-landing/?upn=false&action=getAccessControl&timeout=90
	at org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.checkException(AzureBlobFileSystem.java:1443)
	at org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.getFileStatus(AzureBlobFileSystem.java:652)
	at org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.getFileStatus(AzureBlobFileSystem.java:640)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1760)
	at org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.exists(AzureBlobFileSystem.java:1236)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$4(DataSource.scala:757)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$4$adapted(DataSource.s

In [ ]:
events.show()
print((events.count(), len(events.columns)))
events.printSchema()

In [ ]:
devices.show()
print((devices.count(), len(devices.columns)))
devices.printSchema()

In [ ]:
df = events.join(devices, events["device_id"] == devices["device_id"]) \
    .select(events["*"], devices["browser_type"], devices["os_type"], devices["device_type"])

df.show()